<a href="https://colab.research.google.com/github/alexf05/pclp3/blob/main/TEMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pclp3 tema
# FECHET Alex-Ciprian, 314CA
# JILAVEANU Mihnea, 314CA

Partea I - Fechet Alex
Am ales o problema de regresie liniara,Predicția duratei de somn în funcție de obiceiuri zilnice.
Am optat pentru generarea sintetică a unui dataset care să aibă sens contextual.
Setul de date este compus din 8 coloane: varsta, gen, ore petrecute in fata unui ecran, numarul de cafele consumate, timpul de sport facut (masurat in minute), nivelul de stres, ora de culcare si coloana finala - durata de somn, pe care vom aplica si regresia liniara. Acesta este calculata cu o formula intuitiva, de la 8 ore, scadem proportional cu nr de ore de ecran, nr de cafele, intarzierea orei de culcare, zgomotului, si creste cu practicarea sportului.

In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Setare seed pentru consistență
np.random.seed(42)

# Număr total de exemple
num_rows = 800

# Variabile simulate
varsta = np.random.randint(18, 60, size=num_rows)
gen = np.random.choice(['M', 'F'], size=num_rows)
ore_ecran = np.round(np.random.normal(5, 2, size=num_rows), 1)
ore_ecran = np.clip(ore_ecran, 0, 12)

cafea = np.random.poisson(1.5, size=num_rows)
minute_sport = np.random.randint(0, 120, size=num_rows)
nivel_stres = np.random.choice(['scazut', 'mediu', 'ridicat'], size=num_rows, p=[0.3, 0.5, 0.2])

ora_culcare = np.random.normal(23, 1.5, size=num_rows)  # ex: între 20–26 natural
ora_culcare = np.clip(ora_culcare, 20, 26)              # limităm între 20 și 26
ora_culcare = np.where(ora_culcare >= 24, ora_culcare - 24, ora_culcare)
ora_culcare = np.around(ora_culcare, 1)
zgomot = np.random.normal(60, 80, size=num_rows)

# Simulare logică a duratei somnului (variabila țintă)
durata_somn = (
    8
    - 0.3 * ore_ecran
    - 0.4 * cafea
    + 0.01 * minute_sport
    - 0.2 * (ora_culcare - 22)
    - 0.5 * (zgomot > 70)
)
durata_somn = np.round(np.clip(durata_somn, 3.5, 10), 1)

# Creăm DataFrame
df = pd.DataFrame({
    'varsta': varsta,
    'gen': gen,
    'ore_ecran': ore_ecran,
    'cafea': cafea,
    'minute_sport': minute_sport,
    'nivel_stres': nivel_stres,
    'ora_culcare': ora_culcare,
    'durata_somn': durata_somn
})

#stimulam lipsuri
total_cells = df.shape[0] * df.shape[1]
num_missing = int(total_cells * 0.05)

for i in range(num_missing):
    row = np.random.randint(0, df.shape[0])  # rând random
    col = np.random.randint(0, 7)

    df.iat[row, col] = np.nan




# Salvăm într-un CSV complet
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train.to_csv("train_dataset.csv", index=False)
df_test.to_csv("test_dataset.csv", index=False)
X = df.iloc[:, 0:7]
y = df.iloc[:, 7]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)





In [32]:
print("Valori lipsă în X_train:\n", X_train.isnull().sum())
print("\nValori lipsă în X_test:\n", X_test.isnull().sum())

# Calculăm media pentru fiecare coloană numerică din X_train
mean_values = X_train.mean(numeric_only=True)

# Înlocuim valorile lipsă în X_train și X_test cu media din X_train
X_train_filled = X_train.fillna(mean_values)
X_test_filled = X_test.fillna(mean_values)

Valori lipsă în X_train:
 varsta          37
gen             31
ore_ecran       46
cafea           32
minute_sport    41
nivel_stres     30
ora_culcare     33
dtype: int64

Valori lipsă în X_test:
 varsta          10
gen              9
ore_ecran       10
cafea           11
minute_sport     8
nivel_stres      3
ora_culcare      8
dtype: int64


In [ ]:


# import seaborn as sns
# import matplotlib.pyplot as plt

# # Histograme pentru numerice
# X_train.select_dtypes(include=['int64', 'float64']).hist(bins=20, figsize=(15, 10))
# plt.tight_layout()
# plt.show()

# # Countplot pentru categorice (exemplu)
# for col in X_train.select_dtypes(include=['object']).columns:
#     plt.figure(figsize=(6, 4))
#     sns.countplot(data=X_train, x=col)
#     plt.title(f"Distribuție pentru {col}")
#     plt.xticks(rotation=45)
#     plt.tight_layout()
#     plt.show()

# # Boxplot pentru fiecare coloană numerică
# for col in X_train.select_dtypes(include=['int64', 'float64']).columns:
#     plt.figure(figsize=(6, 4))
#     sns.boxplot(x=X_train[col])
#     plt.title(f"Boxplot pentru {col}")
#     plt.show()


# corr = X_train.select_dtypes(include=['int64', 'float64']).corr()

# plt.figure(figsize=(10, 8))
# sns.heatmap(corr, annot=True, cmap='coolwarm')
# plt.title("Matrice de corelații")
# plt.show()


# # Violin plot pentru numerică vs target
# for col in X_train.select_dtypes(include=['int64', 'float64']).columns:
#     plt.figure(figsize=(6, 4))
#     sns.violinplot(x=y_train, y=X_train[col])
#     plt.title(f"Distribuția {col} în funcție de target")
#     plt.show()


# # Scatter plot între fiecare coloană și target
# for col in X_train.select_dtypes(include=['int64', 'float64']).columns:
#     plt.figure(figsize=(6, 4))
#     sns.scatterplot(x=X_train[col], y=y_train)
#     plt.title(f"Relație {col} - target")
#     plt.show()